# Linear models and Regression

This worksheet will guide you to using regressionmodels. 

When you have classes of data (example cats and dogs...) you can classify these using classifiers. But what if you are trying to predict a value on a scale... For example you might want to predict how much a house costs based on its size. You cannot predict between a class...

This tutorial will guide you through 
- downloading an open dataset
- setting up various regression models
- tuning hyperparameters to get the best outcome

You will need to make sure you have installed the libraries of 
- opendatasets
- sklearn

In [ ]:
!pip install opendatasets
!pip install -U scikit-learn

In [2]:
import numpy as np #library for using large arrays in python
import pandas as pd #library for playing with datasets

############# regression libraries
import opendatasets as od


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## importing a dataset
Firstly take a look at Kaggle https://www.kaggle.com/ which is a great platform for people posting datasets. Have a look round, for now we wll be using a specific dataset but later on you should download datasets that interests you.

In [ ]:
#to import the data we will use the open datasets library 
dataset="https://www.kaggle.com/datasets/shree1992/housedata"
#we place the link copied from Kaggle into the parameter of the download function
od.download(dataset)

## playing with our data

Great now we have downloaded a dataset... but can we just plug this into a model? No! We have to look at our data, how it is formatted and how this looks. The data is downloaded in a folder... 